In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # /data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [14]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from fastai.tabular.all import *
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn import metrics 
from sklearn.metrics import mean_absolute_error, mean_squared_error


from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeClassifier, plot_tree


In [3]:


test_set = pd.read_csv('https://github.com/aliochilov/Aviachipta_narxini_bashorat_qilish/blob/main/test_data.csv?raw=True')
train_set = pd.read_csv('https://github.com/aliochilov/Aviachipta_narxini_bashorat_qilish/blob/main/train_data.csv?raw=True')
y = train_set['price'].copy()
train_set.drop(['id','flight'],inplace=True ,axis=1)

id = pd.DataFrame(test_set['id'])

test_set.drop(['id','flight'], inplace=True ,axis=1)

tst_df=pd.get_dummies(test_set)
df = pd.get_dummies(train_set)




In [ ]:
from sklearn.model_selection import KFold
kf = KFold(n_splits = 10,shuffle=True)
kf

In [5]:
def get_score(model,X_train, X_test, y_train, y_test):
    model.fit(X_train,y_train)
    X = model.predict(X_test)
    return np.sqrt(mean_squared_error(X, y_test))



In [6]:
dep_var="price"
cont_names, cat_names=cont_cat_split(df, 1, dep_var=dep_var)

In [7]:
scaler=StandardScaler()
df[cont_names]=scaler.fit_transform(df[cont_names])
tst_df[cont_names]=scaler.transform(tst_df[cont_names])

features=tst_df.columns.tolist()

In [20]:
scores_t = []
scores_xgb = []
scores_r = []
for fold,(train_idxs,valid_idxs) in enumerate(kf.split(X=df)):
    train_xs, train_y=df.iloc[train_idxs][features], df.iloc[train_idxs][dep_var]
    valid_xs, valid_y=df.iloc[valid_idxs][features], df.iloc[valid_idxs][dep_var]
    tst_xs=tst_df[features]
    
    xgb_model = XGBRegressor()
    xgb_model.fit(train_xs, train_y)
    scores_xgb.append(xgb_model.predict(tst_xs))
    
    rf = RandomForestRegressor()
    rf.fit(train_xs, train_y)
    scores_r.append(rf.predict(tst_xs))
    

    
    print(fold,
        get_score(XGBRegressor(),train_xs ,valid_xs,train_y,valid_y),
        get_score(RandomForestRegressor(n_estimators=43),train_xs ,valid_xs,train_y,valid_y))
 
    

0 3945.481782303342 3828.5101558952338
1 4051.3673822358405 3794.0400775952
2 3953.638736908155 3660.8379391072785
3 4086.640815345758 4009.945444622075
4 3973.6409106592664 3805.7776243712283
5 3887.603823721331 3757.8074384390106
6 3652.5417875126777 3519.7919911085173
7 3938.610939858212 3873.3337122464595
8 3889.6021249629293 3796.4336933228933
9 4116.503232255277 3949.2700352916618


In [18]:
    param_grid = {'n_estimators': np.arange(1, 50)}

    knn_gscv = GridSearchCV(rf, param_grid, cv=5)

    knn_gscv.fit(train_xs, train_y)

GridSearchCV(cv=5, estimator=RandomForestRegressor(),
             param_grid={'n_estimators': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49])})

In [19]:
knn_gscv.best_params_


{'n_estimators': 43}

In [21]:
scores_xgb = np.mean(np.column_stack(scores_xgb), axis=1)
scores_r = np.mean(np.column_stack(scores_r), axis=1)

In [22]:
sub = (scores_xgb+scores_r)/2
sub = {
    'id': id.id.values,
    'Price': sub
}
solution = pd.DataFrame(sub)
solution.to_csv('submission.csv',index=False)